In [4]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [(datetime.datetime.fromisoformat(res['time'][:19])).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        
        #temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).hour]) #時間を追加
                
        #temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).day])  #日を追加
        
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data




def select_data(year_from = 2000, month_from = 1, 
                year_to = 2020, month_to = 9,
                money='USD_JPY', foot = 'H1'):

    filename = str(year_from)+str(month_from)+"to"+str(year_to)+str(month_to)+"_"+money+"_"+foot+ ".csv"
    
    year_months = []

    for i in range(year_from,year_to+1):
        for j in range(month_from,month_to+1):
            temp = [i, j] 
            year_months.append(temp)
    
    
    all_data = []
    for year, month in year_months:
        NY = datetime.timedelta(hours=0)
        date_from = datetime.datetime(year, month, 1) +NY
        date_to = date_from + relativedelta(months=+1, day=1) +NY

        ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
        month_data = oandaJsonToPythonList(ret)#取得したデータを格納

        all_data.extend(month_data)#データを追加

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)


    df.columns = ['Datetime',
                  'Volume', 'Open', 'High', 'Low', 'Close']
    df = df.set_index('Datetime')
    
    df.to_csv(filename, mode = 'w')
    print(filename)
    return df

select_data(year_from = 2010, month_from = 1, 
                year_to = 2020, month_to = 10,
                money='USD_JPY', foot = 'D')

20101to202010_USD_JPY_D.csv


Volume     Open     High      Low    Close
Datetime                                                    
2009-12-31 22:00       3   93.069   93.069   93.047   93.047
2010-01-02 22:00     409   92.943   93.054   92.925   92.986
2010-01-03 22:00   27586   92.984   93.213   92.186   92.509
2010-01-04 22:00   29222   92.506   92.555   91.258   91.704
2010-01-05 22:00   29730   91.706   92.740   91.526   92.320
...                  ...      ...      ...      ...      ...
2020-10-25 21:00   56784  104.715  105.056  104.656  104.834
2020-10-26 21:00   48351  104.841  104.890  104.389  104.416
2020-10-27 21:00   68187  104.408  104.558  104.114  104.314
2020-10-28 21:00   72141  104.302  104.728  104.025  104.602
2020-10-29 21:00   71631  104.607  104.744  104.125  104.620

[2734 rows x 5 columns]